# Demo on Running Acceptance Tests using Behavior-Driven Development

## Overview
In this notebook, you will generate feature files used to execute acceptance tests for an object-sorting task in the IsaacSim simulator. The computational load for rendering the simulation is handled by a dedicated server, and the rendered environment is visualized in the VNC desktop via a WebRTC client application. Through this notebook, users can interact with the server to execute and visualize the tests.

This demo involves opening the desktop environment as the first step, followed by generating the feature file. Next, the user registers on the server, uploads the feature file, and interacts with the remaining API endpoints to control and analyze the tests.

Once the start command is executed, please wait approximately 20 seconds before starting the WebRTC client application. If it does not load, click on 'View' followed by 'Reload' and input the assigned ip address in the 'Sever:' field

**Open VNC Desktop**

In [ ]:
# before executing this cell, make sure the **Simple** button at the bottom left of the window is unchecked

from utils import display_desktop
display_desktop()

## Steps to generate feature files (when using terminal from VNC)
- Open a terminal, either in the desktop environment or in a separate tab within JupyterLab
- Deactivate conda environment: `conda deactivate`
- Activate the Python virtual environment, where relevant packages are already installed: `source ~/venv/bin/activate`
- Navigate to *bdd-dsl/examples* path: `cd ~/behave-isaac-bdd/bdd-dsl/examples/`
- Execute *generate_bdd_specs.py*: `python3 generate_bdd_specs.py`

Continue the steps from registering user from below 

**import libraries and set server ip**

In [ ]:
import subprocess

# ip address of server used to access its API endpoint
ws_ip_addr="192.168.0.203"

**Generate the feature file**

In [ ]:
%%bash --bg
python3 ~/behave-isaac-bdd/bdd-dsl/examples/generate_bdd_specs.py

**API endpoints**
- `register_user`: Assigns the user to a GPU pod and returns its IP address (used in WebRTC client)
- `status`: Retrieves the current status of the assigned pod
- `upload`: Uploads a .feature file for simulation
- `start`: Starts the simulation on the assigned pod
- `stop`: Stops the simulation
- `logs?n`: Returns the last N lines from the simulation logs
- `logs_stream`: Streams live logs from the simulation
- `release`: Deregisters the user and frees the assigned pod

**Note:**
- The API calls can be executed in any sequence that fits your custom logic
- Release should be called at the end of the session to free up resources
- Each user is assigned 30 minutes of usage time. Once the time expires, the user can register again to continue using the service

**register_user**

In [ ]:
%%bash -s "$ws_ip_addr"
curl http://$1:8000/register_user

**status**

In [ ]:
%%bash -s "$ws_ip_addr"
curl http://$1:8000/status

**upload**

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/upload -F "file=@/home/jovyan/behave-isaac-bdd/bdd-dsl/examples/generated/var__us-sorting.feature"

**start**

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/start

**logs_stream**

In [ ]:
# start log stream in a terminal in VNC desktop
subprocess.Popen([
                    "gnome-terminal",
                     "--",
                     "bash", "-c", f"curl http://{ws_ip_addr}:8000/logs_stream; exec bash"
                 ])

**start WebRTC client application**

In [ ]:
# Update the server ip with the ip assigned above after registering
subprocess.Popen([
                    "gnome-terminal",
                     "--",
                     "bash", "-c", f"webrtc-client --no-sandbox; exec bash"
                 ])

**logs?n**

In [ ]:
%%bash -s "$ws_ip_addr"
curl http://$1:8000/logs?n=10

**stop**

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/stop

**release**

In [ ]:
%%bash -s "$ws_ip_addr"
curl -X POST http://$1:8000/release